In [1]:
import torch
import numpy as np
import wandb

from utils import parse_config, unflatten_dot
from dataset import get_loader
from learner import Learner
from models import ASTPretrainedBigHead, ASTPretrainedSmallHead, RNN, ASTWithWeightedLayerPooling
from types import SimpleNamespace

In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
def main():
    
    wandb.init()
    config = wandb.config
    train_dl = get_loader(config, subset="train")
    valid_dl = get_loader(config, subset="valid")
    
    model = ASTPretrainedSmallHead(n_classes=11)

    learn = Learner(train_dl, valid_dl, model, config)

    learn.fit(4)

In [5]:
import yaml
CONFIG_PATH = "./sweep_config.yaml"
with open(CONFIG_PATH) as file:
    config = yaml.safe_load(file)

sweep_id = wandb.sweep(sweep=config)
wandb.agent(sweep_id, function=main, count=1)

Create sweep with ID: 4p6me4zo
Sweep URL: https://wandb.ai/k-pintaric/uncategorized/sweeps/4p6me4zo


wandb: Agent Starting Run: 4nntz0wk with config:
wandb: 	EPOCHS: 5
wandb: 	batch_size: 8
wandb: 	early_stopping: {'min_delta': 0.0001, 'patience': 3}
wandb: 	learning_rates: {'base_model': {'lr': 1.2878301358694264e-08}, 'classifier': {'lr': 0.02911756816973033}}
wandb: 	loss: {'FocalLoss': {'alpha': -1, 'gamma': 2}}
wandb: 	metrics: ['hamming_score', 'zero_one_score', 'mean_average_precision', 'mean_f1_score']
wandb: 	num_accum: 6
wandb: 	optimizer: {'AdamW': {'weight_decay': 7.181742645634947e-07}}
wandb: 	preprocess: {'PreprocessPipeline': {'target_sr': 22050}}
wandb: 	save_best_model: True
wandb: 	scheduler: {'OneCycleLR': {'anneal_strategy': 'cos', 'pct_start': 0.1939329785206244}}
wandb: 	signal_augments: None
wandb: 	spec_augments: None
wandb: 	train_dir: ./data/processed/all_sync/IRMAS_Training_Data
wandb: 	transforms: {'LogTransform': None, 'Normalize': {'mean': -4.2677393, 'std': 9.1379948}, 'PadCutToLength': {'max_length': 1024}, 'Preemphasis': None, 'Spectrogram': {'hop_len

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/9220 [00:00<?, ?it/s]

Run 4nntz0wk errored: RuntimeError('Caught RuntimeError in DataLoader worker process 0.\nOriginal Traceback (most recent call last):\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop\n    data = fetcher.fetch(index)\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch\n    data = [self.dataset[idx] for idx in possibly_batched_index]\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>\n    data = [self.dataset[idx] for idx in possibly_batched_index]\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/dataset.py", line 52, in __getitem__\n    signal = self.transforms(signal)\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/utils.py", line 149, in __call__\n    input = t(input, *args)\n  File "/nfs/home/912c2e22633dde2e/lumen-irmas/tr